In [1]:
import time
import re
import base64
import logging
from PIL import Image
from io import BytesIO
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime


In [2]:

# Configure logging
logging.basicConfig(filename='selenium_continuous_capture.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Set up WebDriver
driver = webdriver.Chrome()
logging.info("WebDriver initiated.")

url = "http://littlenedr4.local/dashboard/robot/mission_control"
driver.get(url)
logging.info(f"URL loaded: {url}")

try:
    # Allow time for all scripts and dynamic content to load
    time.sleep(30)
    logging.info("Waited 30 seconds for page and scripts to load.")

    while True:
        try:
            elements = driver.find_elements(By.CSS_SELECTOR, '[data-renderer="SensorMsgs_CompressedImage"]')
            if elements:
                element = elements[0]  # Assuming the first element is the correct one
                style = element.value_of_css_property('background-image')
                logging.info(f"Style attribute fetched: {style}")

                url_match = re.search(r'url\("([^"]+)"\)', style)
                if url_match:
                    image_url = url_match.group(1)
                    if image_url.startswith('data:image'):
                        base64_str = image_url.split('base64,')[1]
                        image_data = base64.b64decode(base64_str)
                        image = Image.open(BytesIO(image_data))
                        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")  # Generates timestamp
                        filename = f"image_{timestamp}.jpg"
                        image.save(filename)
                        logging.info(f"Image saved: {filename}")
                    else:
                        logging.warning("No base64 image in URL.")
                else:
                    logging.warning("No URL could be extracted.")
            else:
                logging.warning("No elements found with specified selector.")

            time.sleep(1)  # Delay to avoid overloading and to respect the data update rate

        except Exception as e:
            logging.error(f"Error during image processing: {e}")
            time.sleep(5)  # Wait some time before trying again in case of an error

except KeyboardInterrupt:
    logging.info("Manual interruption received.")

finally:
    driver.quit()
    logging.info("WebDriver session closed.")